In [1]:
#default_exp Clustering

In [2]:
#hide
from nbdev import *
from nbdev.showdoc import show_doc

In [3]:
#export
import os
import sys
import numpy as np
import pandas as pd
import kneed
import itertools
import shapely
import random
from CGAL.CGAL_Alpha_shape_2 import *
from CGAL.CGAL_Kernel import Point_2
from sklearn.cluster import DBSCAN, OPTICS
from sklearn.preprocessing import StandardScaler
from shapely.geometry import LineString
from shapely.ops import polygonize, cascaded_union
from shapely.geometry import box
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.ops import polygonize_full, linemerge, unary_union
from scipy.spatial import cKDTree, Delaunay
import hdbscan
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from HierarchicalGeoClustering.TreeClusters import *

In [4]:
#hide
#export
def get_alpha_shape(point_list):
    """
    Returns a polygon representing the hull of the points sample.
    
    :param list point_list: list list of tuples with samples coordinates.
    
    :returns shapely.Polygon: concave hull shapely polygon
    """
    uni_po = np.unique(point_list, axis=0)
    if len(uni_po) < 3:
        raise ValueError('Alpha Shape needs more than 3 points')
    if set_colinear(uni_po) == True:
        raise ValueError('The set of points can be colinear')

    list_of_points = [Point_2(l[0], l[1]) for l in point_list]

    a = Alpha_shape_2()

    a.make_alpha_shape(list_of_points)
    a.set_mode(REGULARIZED)
    alpha = a.find_optimal_alpha(1).next()
    a.set_alpha(alpha)

    edges = []
    for it in a.alpha_shape_edges():
        edges.append(a.segment(it))

    lines = []
    for e in edges:
        source_p = (e.source().x(), e.source().y())
        target_p = (e.target().x(), e.target().y())
        lines.append(LineString([source_p, target_p]))

    return cascaded_union(list(polygonize(lines)))

In [5]:
#hide
#export
def set_colinear(list_points):
    """
    Check if in the list of points any of triplet of points
    is colinear
    :param list list_points: List of shapely Points
    
    :returns bool: True if all are not colinear 
    """
    for i in itertools.combinations(list_points, 3):
        if collinear(i[0], i[1], i[2]) == False:
            return False
    return True

In [6]:
#hide
#export
def collinear(p1, p2, p3):
    """
    Check if the points are colinear 
    
    :param shapely Point p1: point to chek if is colinear
    
    :param shapely Point p2: point to chek if is colinear
    
    :param shapely Point p3: point to chek if is colinear
    
    :return bool: True if are colinear
    """
    return (p1[1]-p2[1]) * (p1[0]-p3[0]) == (p1[1]-p3[1])*(p1[0]-p2[0])

In [7]:
#hide
#export
def get_segments(points):
    """ 
    Get the segments from a delaunay triangulation
    
    :param points: Point to get Delaunay triangulation and exctract points 
    
    :return edges: 
    """
    TIN = Delaunay(points)
    # list of coordinates for each edge
    edges = []
    for tr in TIN.simplices:
        for i in range(3):
            edge_idx0 = tr[i]
            edge_idx1 = tr[(i+1) % 3]
            edges.append(LineString((Point(TIN.points[edge_idx0]),
                                    Point(TIN.points[edge_idx1]))))

    return edges

In [8]:
#hide
#export 
def get_polygons_buf(lines):
    """
    Obtain the poligons from the lines
    
    :param list lines: List of lines
    
    :returns shapely polygon: the union of the union of 
    edges (Polygon or multypolygon)
    """
    linework = linemerge(lines)
    linework = unary_union(linework)
    result, _, _, _ = polygonize_full(linework)
    result = unary_union(result)
    result = result.buffer(0.0000001)
    return result

In [9]:
#hide
#export 
def labels_filtra(point_points, multy_pol):
    """
    Labels the points in the multy_pol if no polygon contains 
    a point is label as -1
    
    :param shapely MultyPoint point_points: Points to check 
    
    :param multy_pol
    
    :returns np.array: Label array with -1 if is not contained 
    in a polygon
    """
    point_Po = [Point(i) for i in  point_points]
    labels_p=[]
    if type(multy_pol)==shapely.geometry.MultiPolygon :
        for po in point_Po:
            if multy_pol.contains(po):
                for num_pol, poly in enumerate( multy_pol):
                    if poly.contains(po):
                        labels_p.append(num_pol)
                        break
            else:
                labels_p.append(-1)
    elif type(multy_pol)==shapely.geometry.Polygon :
        for po in point_Po:
            if multy_pol.contains(po):
                labels_p.append(0)
            else:
                labels_p.append(-1)
    else:
        raise ValueError('The input is not MultiPolygon or Polygon type')   
    
    return np.array(labels_p)

In [10]:
#hide
#export 
def levels_from_strings(
            string_tag,
            level_str='l_',
            node_str = 'n_',
            **kwargs
            ):
    """
    Returns the levels and the node id using the expected strings 
    that identify the level id and node id
    
    :param str level_str: string for the level
    
    :param str node_str: string for the nodes
    
    
    :returns tuple (levels, nodeid): 
    """
    
    positions = [i.start() for i in re.finditer( level_str, string_tag )]
    levels = [string_tag[i+len(level_str)] for i in positions ]
    nodeid_positions = [i.start() for i in re.finditer( node_str, string_tag )]    
    nodeid = [string_tag[i+len(node_str)] for i in nodeid_positions ]

    return levels, nodeid

In [11]:
#hide
#export 
def get_tag_level_df_labels(df, levels_int ):
    """
    Get the tag for the cluster
    
    :param Pandas.DataFrame df:
    
    :param int levels_int: 
    
    :returns None:
    """
    for i in range(levels_int):
        df['level_'+ str(i) +'_cluster']= df['cluster_id'].apply(lambda l:  level_tag(l,i))

In [12]:
#hide
#export 
def level_tag(list_tags, level_int  ):
    """
    Tags if the are nois or signal
    """
    if len(list_tags)==0:
        return 'noise'
    try:
        return list_tags[level_int]
    except:
        return 'noise'   

In [13]:
#hide
#export 
def get_dics_labels(tree_or, tree_res, level_get):
    """
    Obtains a list of dictionaries to retag the original tree_tag with their 
    correspondance in the tree_res on level level_get +1
    
    :param tree_or:
    
    :param tree_res:
    
    :param level_get:
    
    :param return list:
    """
    dic_list_levels= []
    for i in range(level_get):
        dic_level_df = get_label_clusters_df(tree_or, tree_res, i)
        ## Eliminate the clusters with nan  
        dic_level_df.dropna(axis=0, subset=['Sim_cluster'], inplace=True)
        
        dic_lev  = dic_level_df['Sim_cluster'].to_dict()
        dic_list_levels.append({'level_ori':'level_'+str(i)+'_cluster', 'dict': dic_lev})
    return dic_list_levels

In [14]:
#hide
#export

def get_label_clusters_df(tree_1, tree_2, level_int):
    """
    Obtains the dataframe with the label 
    
    :param TreeClusters tree_1: 
    
    :param TreeClusters tree_2:
    
    :param int level_int:
    
    :reutrns Pandas.DataFrame df_level_clus: 
    """
    level_all = tree_1.get_level(level_int)
    df_level_clus = pd.DataFrame(level_all, columns=['Clusters'])
    df_level_clus['Area'] = df_level_clus['Clusters'].apply(lambda l: l.polygon_cluster.area)
    df_level_clus['Name'] = df_level_clus['Clusters'].apply(lambda l: l.name)
    
    df_level_clus['Sim_cluster'] = df_level_clus['Clusters'].apply(lambda l: get_mini_jaccars(l, tree_2,level_int+1)) ###### Como se hacen las clusterizaciones se debe usar el siguiente nivel
    
    #print('', df_level_clus['Sim_cluster'].dtype)
    df_level_clus= df_level_clus.sort_values(by ='Area', ascending=False)
    df_level_clus['Sim_cluster'] = (df_level_clus['Sim_cluster']
                                    .where(~df_level_clus.duplicated(subset=['Sim_cluster']), None))
    #print(df_level_clus['Sim_cluster'].dtype)
    level_2= tree_2.get_level(level_int+1) 
    df_level_clus['Sim_cluster_name'] =(df_level_clus['Sim_cluster']
                                         .astype('int32', errors='ignore')
                                         .replace({np.nan: ''})
                                         .apply(lambda l:  level_2[int(l)].name if l !=''  else None) )
    
    return df_level_clus

In [15]:
#hide
#export 
def get_mini_jaccars(cluster, tree_2, level_int):
    """
    Find the most simmilar cluster in the tree_2 at level level_int
    returns int the index of the most similar polygon in the level
    """
    tree_2_level= tree_2.get_level(level_int)
    Jaccard_i= [jaccard_distance(cluster.polygon_cluster, j.polygon_cluster) for j in tree_2_level]  
    
    valu_min = Jaccard_i.index( min(Jaccard_i))
    return valu_min

In [16]:
#hide
#export 
def jaccard_distance(p1, p2):
    """
    Computes the Jaccard similarity measuremen between two polygons.
    
    param: p1 shapely Poligon 
    param: p2 shapely Poligon 
    return float Jaccard distance
    """
    intersection_area = p1.intersection(p2).area  
    #print(intersection_area)
    jacc= 1 - (intersection_area)/(p1.area + p2.area - intersection_area)
    return jacc

In [17]:
#hide
#export 
def mod_cid_label(dic_label):
    """
    
    """
    dic_label={str(k):str(v) for k,v in dic_label.items()} 
    dic_label['noise'] = 'noise'
    return dic_label

In [18]:
#hide
#export
def retag_originals(df_fram_or , df_results, tag_original, tag_results, dic_tag_or_res):
    """
    Retags the labels in the df_fram_or using the dictionary dic_tag_or_res to match 
    the tags with the corresponding tag in the df_result and all the labels that are
    not in the dictionary generate a new tag fo them. 
    
    
    :param Pandas.DataFrame df_fram_or
    
    :param Pandas.DataFrame df_results
    
    :param tag_original
    
    :param tag_results
    
    :param Pandas.DataFrame dic_tag_or_res
    
    """
    tag_plus=  len(df_results[tag_results].unique()) +100  - len(df_results[tag_results].unique())%100
    df_fram_or['re_tag_'+str(df_results.name)+'_'+tag_original] = df_fram_or[tag_original].apply(lambda l: dic_tag_or_res[l] if l in dic_tag_or_res.keys() else   str(int(l) +tag_plus) )

# Clustering 

In this module all the clustering methods wrap or implemented (Natural cities, DBSCAN, OPTICS, HDBSCAN, and adap_DBSCAN) all with the intention to have the same input and output. 

A recursive function is implemented to obtain the cluster iterative using the output as the new input. To select the method to use only a string is needed.


In [19]:
#export
def clustering(
            t_next_level_2,
            level=None,
            algorithm='dbscan',
            **kwargs
    ):
    """Function to get the clusters for single group by
    
    :param t_next_level_2 Dictionary with the points to compute the
            cluster
    :param level:  None Level to compute (Default None)
    
    :param str algorithm : Algorithm type is supported (Default= 'dbscan')
    
    :param int min_points_cluster:  minimun number of point to consider a cluster(Default 50)
    
    :param double eps: Epsilon parameter In case is needed
    
    :param bool return_noise: To return the noise (Default False)
    
    :param bool verbose: Printing (Dafault False)  
    
    :returns list t_next_level_n: A list with dictionaries with the points, the parent, and nois 
    """
    verbose= kwargs.get('verbose',False)
    min_points = kwargs.get( 'min_points_cluster', 50)
    ret_noise= kwargs.get('return_noise', False)
    eps = kwargs.get('eps',0.8)  # Epsilon value to dbscan
    t_next_level_n = []
    if level == None:
        level = 0

    for li_num, cluster_list_D in enumerate(t_next_level_2):
        cluster_list = cluster_list_D['points']
        cluster_list_pa = cluster_list_D['parent']
        if verbose:
            print("Size cluster list: ", len(cluster_list))
            
        for c_num, cluster in enumerate(cluster_list):
            if verbose:
                print("Size cluster: ", len(cluster))
                print('Algorithm: ', algorithm)

            if len(cluster) > 5:
                if algorithm == 'dbscan':
                    if verbose:
                        print("Epsilon Value: ", eps)
                    tmp = compute_dbscan(cluster,
                                 eps_DBSCAN = eps,
                                 debugg=verbose,
                                  **kwargs)
                    if ret_noise:
                        noise_points = tmp[1]
                        tmp =  tmp[0]
                
               
                elif algorithm == 'hdbscan':
                    tmp = compute_hdbscan(cluster,
                                **kwargs)
                    if ret_noise:
                        noise_points = tmp[1]
                        tmp =  tmp[0]
                ##########  
                elif algorithm == 'adaptative_DBSCAN':
                    #### If the number of cluster is too small 
                    
                    tmp = adaptative_DBSCAN(cluster, **kwargs)
                    if ret_noise:
                        noise_points = tmp[1]
                        tmp =  tmp[0]

                elif algorithm == 'optics':
                    tmp = compute_OPTICS(cluster,
                                eps_OPTICS = eps,
                                **kwargs)
                    if ret_noise:
                        noise_points = tmp[1]
                        tmp =  tmp[0]
                ##########  
                elif algorithm == 'natural_cities':
                    tmp = compute_Natural_cities(cluster,
                                **kwargs)
                    if ret_noise:
                        noise_points = tmp[1]
                        tmp =  tmp[0]
                ##########  
                else:
                    raise ValueError('Algorithm must be dbscan or hdbscan')
                    # sys.exit("1")
                
                
                
                if verbose:
                    print("The number of resulting clusters is : ", len(tmp))
                if ret_noise:
                    dic_clos = {'points': tmp,
                           'parent': cluster_list_pa + '_L_'+str(level) +
                            '_l_' + str(li_num) + '_c_'+str(c_num), 
                            'noise_points':noise_points
                    }
                else:
                    dic_clos = {'points': tmp, 'parent': cluster_list_pa +
                            '_L_'+str(level) + '_l_' + str(li_num) + '_c_'+str(c_num)}
                
                t_next_level_n.append(dic_clos)
            else:
                if ret_noise:
                    dic_clos = {'points': [],
                           'parent': cluster_list_pa + '_L_'+str(level) +
                            '_l_' + str(li_num) + '_c_'+str(c_num), 
                            'noise_points':cluster
                    }
                else:
                    dic_clos = {'points': [], 'parent': cluster_list_pa +
                            '_L_'+str(level) + '_l_' + str(li_num) + '_c_'+str(c_num)}
                t_next_level_n.append(dic_clos)
    
    return t_next_level_n

In [20]:
#hide
### Pruebas 



In [21]:
show_doc(clustering)

<h4 id="clustering" class="doc_header"><code>clustering</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>clustering</code>(**`t_next_level_2`**, **`level`**=*`None`*, **`algorithm`**=*`'dbscan'`*, **\*\*`kwargs`**)

Function to get the clusters for single group by

:param t_next_level_2 Dictionary with the points to compute the
        cluster
:param level:  None Level to compute (Default None)

:param str algorithm : Algorithm type is supported (Default= 'dbscan')

:param int min_points_cluster:  minimun number of point to consider a cluster(Default 50)

:param double eps: Epsilon parameter In case is needed

:param bool return_noise: To return the noise (Default False)

:param bool verbose: Printing (Dafault False)  

:returns list t_next_level_n: A list with dictionaries with the points, the parent, and nois 

In [22]:
#export
def recursive_clustering(
                this_level,  # Dictionary with Points
                to_process,  # levels to process
                cluster_tree,  # to store the clusters
                level = 0,  # current level
                algorithm ='dbscan',  # Algorithm to use
                **kwargs
               ):
    """
    Performs the recursive clustering.
    Calls compute_dbscan for each
    list of clusters keepen the structure and then calls itself
    until no more clusters satisfy the condition
        
    :param dict this_level: level is the current level 
    
    :param int to_process: the max level to process
    
    :param double eps: The epsilon parameter distance to pass to the needed algorithm 
    
    :param list cluster_tree : list of list to insert the levels 
    
    :param bool verbose : To print 
    
    :param double decay: In the use of dbscan the deacy parameter to reduce eps
    
    :param int min_points_cluster: The min point for each cluster to pass to algorithm
    
    :param str algorithm:  The string of the algorithm name to use
    """

    verbose= kwargs.get('verbose',False)
    min_points = kwargs.get( 'min_points_cluster', 50)
    decay = kwargs.get('decay', 0.7)
    eps = kwargs.get('eps' ,0.8)  # Epsilon distance to DBSCAN parameter
    tmp = None

    if level == 0:
        kwargs['eps'] = eps
    else:
        kwargs['eps'] = eps  * decay

    cluster_result_polygons = []
    if level > to_process:
        if verbose:
            print('Done clustering')
        return
    ######## Get the clusters for the current list of points 
    all_l = clustering(
                    this_level,
                    level=level,
                    algorithm=algorithm,
                    
                    **kwargs
                    )
    ##########

    cluster_tree.append(all_l)
    cluster_n = 0
    for i in all_l:
        cluster_n += len(i['points'])
    if verbose:
        print('At level ', level, ' the number of lists are ',
              len(all_l), ' with ', cluster_n, 'clusters')
    level += 1
    if len(all_l) > 0:
        return recursive_clustering(all_l, 
                               to_process=to_process,
                               cluster_tree=cluster_tree,
                               level= level,
                               algorithm=algorithm,
                               **kwargs
                               )
    else:
        if verbose:
            print('done clustering')
        return

In [23]:
#hide
### Pruebas 

In [24]:
show_doc(recursive_clustering)

<h4 id="recursive_clustering" class="doc_header"><code>recursive_clustering</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>recursive_clustering</code>(**`this_level`**, **`to_process`**, **`cluster_tree`**, **`level`**=*`0`*, **`algorithm`**=*`'dbscan'`*, **\*\*`kwargs`**)

Performs the recursive clustering.
Calls compute_dbscan for each
list of clusters keepen the structure and then calls itself
until no more clusters satisfy the condition
    
:param dict this_level: level is the current level 

:param int to_process: the max level to process

:param double eps: The epsilon parameter distance to pass to the needed algorithm 

:param list cluster_tree : list of list to insert the levels 

:param bool verbose : To print 

:param double decay: In the use of dbscan the deacy parameter to reduce eps

:param int min_points_cluster: The min point for each cluster to pass to algorithm

:param str algorithm:  The string of the algorithm name to use

In [25]:
#export 
def recursive_clustering_tree(dic_points_ori, **kwargs):
    """
    Obtaing the recursive tree using a specific algorithm
    """
    levels_clustering= kwargs.get('levels_clustering',4)
    algorithm = kwargs.get('algorithm','dbscan')
    verbose = kwargs.get('verbose', False)
    cluster_tree = []
    recursive_clustering([dic_points_ori],  # Dictionary with Points
                levels_clustering,  # levels to process
                cluster_tree,  # to store the clusters
                level=0,  # current level
                algorithm = algorithm,  # Algorithm to use
                return_noise = True,
                verbose=verbose
                )
    tree_clus= get_tree_from_clustering(cluster_tree)
    tree_from_clus= TreeClusters()
    tree_from_clus.levels_nodes = tree_clus
    tree_from_clus.root= tree_from_clus.levels_nodes[0][0]   
    return tree_from_clus

## Clustering Algorithms
A wapper functions to obtain the clusterizations 

In [26]:
#export
def compute_dbscan(cluster,  **kwargs):
    
    """ 
    Sklearn DBSCAN wrapper.
    
    :param cluster: a (N,2) numpy array containing the obsevations

    :returns list with numpy arrays for all the clusters obtained
    """
    eps = kwargs.get( 'eps_DBSCAN',.04)
    debugg= kwargs.get( 'debugg',False)
    min_samples= kwargs.get( 'min_samples',50)
    ret_noise = kwargs.get('return_noise', False)
    # Standarize sample
    scaler = StandardScaler()
    cluster = scaler.fit_transform(cluster)
    if debugg:
        print('epsilon distance to DBSCAN: ', eps)
        print("min_samples to DBScan: ", min_samples )
        print("Number of points to fit the DBScan: ",cluster.shape[0])

    db = DBSCAN(eps=eps, min_samples=min_samples).fit(cluster)  # Check if can be run with n_jobs = -1
    
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    l_unique_labels = len(set(labels)) - (1 if -1 in labels else 0)
    unique_labels = set(labels) 
    cluster = scaler.inverse_transform(cluster)
    clusters = []
    if debugg:
        print('Number of clusters:' ,l_unique_labels)
    
    for l in unique_labels:
        if l != -1:
            class_member_mask = (labels == l)
            clusters.append(cluster[class_member_mask])
        elif l == -1 and debugg == True:
            class_member_mask = (labels == l)
            print("Muestras consideradas ruido: ",  sum(class_member_mask))
    
    if ret_noise == True:
        class_member_mask = (labels == -1)
        return clusters, cluster[class_member_mask]
    
    return clusters

In [27]:
#hide
### Pruebas 


In [28]:
show_doc(compute_dbscan)

<h4 id="compute_dbscan" class="doc_header"><code>compute_dbscan</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>compute_dbscan</code>(**`cluster`**, **\*\*`kwargs`**)

Sklearn DBSCAN wrapper.

:param cluster: a (N,2) numpy array containing the obsevations

:returns list with numpy arrays for all the clusters obtained

In [29]:
#export
def adaptative_DBSCAN(points2_clusters ,
                **kwargs):
    """
    The function use the knee and average to obtain a good value for epsilon and use 
    DBSCAN to obtain the clusters
    
    :param list Points points2_clusters: Point to clusterize  
    
    :param int max_k: = (Default = len(points2_clusters)*.1)
    
    :param int  min_k: (Default =50)
    
    :param int step_k: (Default = 50)
    
    :param int leaf_size: (Default = 50)
    
    :param bool scale_points: (Default = True)
    
    :param bool debugg: (Default = False)
    
    :param bool ret_noise:  (Default = True)
    
    :returns list : list of cluster. If ret_noise = True return tuple list of cluter and noise 
    """
    max_k = kwargs.get('max_k', int(len(points2_clusters)*.1))
    min_k = kwargs.get('min_k', 50)
    step_k = kwargs.get('step_k', 50)
    leaf_size =  kwargs.get('leaf_size',50)
    scale_points= kwargs.get('scale_points',True)
    debugg = kwargs.get('verbose',False)
    ret_noise = kwargs.get('return_noise', True)
    ###### Se tienen que hacer algunos cambios para cuando
    #  los clusters son menores a los minimos establecidos previemente
    
    ##### Establecer los minimos posibles 
    if max_k > len(points2_clusters):
        raise ValueError('The max_k value is too large for the number of points')
    
    if min_k >  len(points2_clusters):
        print('The min_k value is too large for the number of points returns empty clusters')
        if ret_noise == True:
            return [] , points2_clusters
        else:
            return []
    
    if step_k > len(points2_clusters):
        raise ValueError('The step_k value is too large for the number of points')

    
    if min_k == max_k:
        print('min_k reset to obtain at least 1 value')
        min_k = max_k-1

    if scale_points ==True:
        scaler = StandardScaler()
        points_arr = scaler.fit_transform(points2_clusters)
    else:
        points_arr = points2_clusters
    
    kdt=  cKDTree(points_arr, leafsize=leaf_size)
    lits_appe_all_aver=[]
    for j in range( min_k, max_k, step_k ):
        dist_va, ind = kdt.query(points_arr, k=j, n_jobs =-1) 
        non_zero =  dist_va[:, 1:]
        non_zero = np.ndarray.flatten(non_zero)
        non_zero = np.sort(non_zero)
        lis_aver_k=[]
        for i in range(int(non_zero.shape[0]/(j-1)) -1):
            lis_aver_k.append(np.average(non_zero[i*(j-1):(i+1)*(j-1)]))

        average_arr= np.array(lis_aver_k)
        kneedle_1_average = kneed.KneeLocator(
                range(average_arr.shape[0]),
                average_arr,
                curve="convex",## This should be the case since the values are sorted 
                direction="increasing", ## This should be the case since the values are sorted incresing
                online=True, ### To find the correct knee the false returns the first find 
        )
        epsilon= kneedle_1_average.knee_y
        min_point = kneedle_1_average.knee
        #### We take the average never the less
        
        lits_appe_all_aver.append({ 'k':j,
                    'Epsilon':epsilon,
                    'value':min_point})
    
    #### Check if the list is empty
    if len(lits_appe_all_aver) ==0:
        if debugg:
            print('DBSCAN')
            print('Using 0.6 as epsilon and 20 as Minpoints')
        db_scan= DBSCAN(eps=0.6, min_samples=20).fit(points_arr)
    else:
        df_all_average= pd.DataFrame(lits_appe_all_aver)
        max_epsi_all_average= df_all_average['Epsilon'].max()
        if debugg:
            print('Valor de epsion  : ', max_epsi_all_average)
        db_scan= DBSCAN(eps=max_epsi_all_average, min_samples=min_k).fit(points_arr)
    
    ####Get the clusters
    core_samples_mask = np.zeros_like(db_scan.labels_, dtype=bool)
    core_samples_mask[db_scan.core_sample_indices_] = True
    labels = db_scan.labels_
    unique_labels = set(labels)
    if scale_points ==True:
        points_ret = scaler.inverse_transform(points_arr)
    else:
        points_ret = points_arr
    clusters = []
    for l in unique_labels:
        if l != -1:
            class_member_mask = (labels == l)
            clusters.append(points_ret[class_member_mask])
        elif l == -1 and debugg == True:
            class_member_mask = (labels == l)
            print("Muestras consideradas ruido: ",  sum(class_member_mask))

    if ret_noise == True:
        class_member_mask = (labels == -1)
        return clusters, points_ret[class_member_mask]

    return clusters

In [30]:
show_doc(adaptative_DBSCAN)

<h4 id="adaptative_DBSCAN" class="doc_header"><code>adaptative_DBSCAN</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>adaptative_DBSCAN</code>(**`points2_clusters`**, **\*\*`kwargs`**)

The function use the knee and average to obtain a good value for epsilon and use 
DBSCAN to obtain the clusters

:param list Points points2_clusters: Point to clusterize  

:param int max_k: = (Default = len(points2_clusters)*.1)

:param int  min_k: (Default =50)

:param int step_k: (Default = 50)

:param int leaf_size: (Default = 50)

:param bool scale_points: (Default = True)

:param bool debugg: (Default = False)

:param bool ret_noise:  (Default = True)

:returns list : list of cluster. If ret_noise = True return tuple list of cluter and noise 

In [31]:
#export
def compute_hdbscan(points2_clusters,  **kwargs):
    
    """
    HDBSCAN wrapper.
    
    :param np.array cluster: a (N,2) numpy array containing the obsevations
    
    :returns:  list with numpy arrays for all the clusters obtained
    """
    
    scale_points= kwargs.get('scale_points',True)
    debugg = kwargs.get('verbose',False)
    ret_noise = kwargs.get('return_noise', True)
    min_cluster = kwargs.get('min_cluster', 20)
    if scale_points ==True:
        scaler = StandardScaler()
        points_arr = scaler.fit_transform(points2_clusters)
    else:
        points_arr = points2_clusters

    db = hdbscan.HDBSCAN( ).fit(points_arr)
    core_samples_mask = np.full_like(db.labels_, True, dtype=bool)
    labels = db.labels_
    l_unique_labels = len(set(labels)) - (1 if -1 in labels else 0)
    unique_labels = set(labels)
    if debugg:
        print('total number of clusters: ', len(unique_labels)) 
    if scale_points ==True:
        points_ret = scaler.inverse_transform(points_arr)
    else:
        points_ret = points_arr
    clusters = []

    for l in unique_labels:
        if l != -1:
            class_member_mask = (labels == l)
            clusters.append(points_ret[class_member_mask])
        elif l == -1 and debugg == True:
            class_member_mask = (labels == l)
            print("Muestras consideradas ruido: ",  sum(class_member_mask))

    if ret_noise == True:
        class_member_mask = (labels == -1)
        return clusters, points_ret[class_member_mask]

    return clusters

In [32]:
show_doc(compute_hdbscan)

<h4 id="compute_hdbscan" class="doc_header"><code>compute_hdbscan</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>compute_hdbscan</code>(**`points2_clusters`**, **\*\*`kwargs`**)

HDBSCAN wrapper.

:param np.array cluster: a (N,2) numpy array containing the obsevations

:returns:  list with numpy arrays for all the clusters obtained

In [33]:
#export
def compute_OPTICS(points2_clusters,  **kwargs):
    
    """ OPTICS wrapper.
    :param np.array cluster: a (N,2) numpy array containing the obsevations
    :returns:  list with numpy arrays for all the clusters obtained
    """

    scale_points= kwargs.get('scale_points',True)
    debugg = kwargs.get('verbose',False)
    ret_noise = kwargs.get('return_noise', True)
    min_samples= kwargs.get( 'min_samples',5)
    eps_optics = kwargs.get('eps_optics', None)
    n_jobs = kwargs.get('num_jobs',None)
    xi= kwargs.get('xi',None)
    algorithm_optics= kwargs.get('algorithm_optics','kd_tree')

    if scale_points ==True:
        scaler = StandardScaler()
        points_arr = scaler.fit_transform(points2_clusters)
    else:
        points_arr = points2_clusters


    db = OPTICS(min_samples = min_samples,eps= eps_optics, n_jobs= n_jobs).fit(points2_clusters)
    core_samples_mask = np.full_like(db.labels_, True, dtype=bool)
    labels = db.labels_
    l_unique_labels = len(set(labels)) - (1 if -1 in labels else 0)
    unique_labels = set(labels)
    if debugg:
        print('total number of clusters: ', len(unique_labels)) 
    if scale_points ==True:
        points_ret = scaler.inverse_transform(points_arr)
    else:
        points_ret = points_arr
    clusters = []

    for l in unique_labels:
        if l != -1:
            class_member_mask = (labels == l)
            clusters.append(points_ret[class_member_mask])
        elif l == -1 and debugg == True:
            class_member_mask = (labels == l)
            print("Muestras consideradas ruido: ",  sum(class_member_mask))

    if ret_noise == True:
        class_member_mask = (labels == -1)
        return clusters, points_ret[class_member_mask]

    return clusters

In [34]:
show_doc(compute_OPTICS)

<h4 id="compute_OPTICS" class="doc_header"><code>compute_OPTICS</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>compute_OPTICS</code>(**`points2_clusters`**, **\*\*`kwargs`**)

OPTICS wrapper.
:param np.array cluster: a (N,2) numpy array containing the obsevations
:returns:  list with numpy arrays for all the clusters obtained

In [35]:
#export 
def compute_Natural_cities(points2_clusters,  **kwargs):
    
    """
    Compute Natural cities clustering
    
    :param np.array points2_clusters: a (N,2) numpy array containing the obsevations
    
    :returns: list with numpy arrays for all the clusters obtained
    """
    ### The function is in acordance with the all the previus functions
    scale_points= kwargs.get('scale_points',True)
    debugg = kwargs.get('verbose',False)
    ret_noise = kwargs.get('return_noise', True)

    if scale_points ==True:
        scaler = StandardScaler()
        points_arr = scaler.fit_transform(points2_clusters)
    else:
        points_arr = points2_clusters

    edges= get_segments(points_arr)
    lenght_av  =  np.average(np.array([i.length for i in edges ]))
    edges = [i for i in edges  if i.length < lenght_av]
    polygons_natural_cities=  get_polygons_buf(edges)
    if debugg:
        if type(polygons_natural_cities)==shapely.geometry.MultiPolygon:
            print('Resulting number of polygons: ', len(polygons_natural_cities))

        elif type(polygons_natural_cities)==shapely.geometry.Polygon:
            print('Only 1 polygon: ')
        else:
            print('The result is not a Polygon or Multipolygon')
    labels_points = labels_filtra(points_arr, polygons_natural_cities)
    core_samples_mask = np.full_like(labels_points, True, dtype=bool)
    l_unique_labels = len(set(labels_points)) - (1 if -1 in labels_points else 0)
    unique_labels = set(labels_points)
    
    if debugg:
        print('total number of clusters: ', len(unique_labels)) 
    #### recover
    if scale_points ==True:
        points_ret = scaler.inverse_transform(points_arr)
    else:
        points_ret = points_arr

    
    clusters = []
    for l in unique_labels:
        if l != -1:
            class_member_mask = (labels_points == l)
            clusters.append(points_ret[class_member_mask])
        elif l == -1 and debugg == True:
            class_member_mask = (labels_points == l)
            print("Point conscider noise: ",  sum(class_member_mask))

    if ret_noise == True:
        class_member_mask = (labels_points == -1)
        return clusters, points_ret[class_member_mask]

    return clusters

In [36]:
show_doc(compute_Natural_cities)

<h4 id="compute_Natural_cities" class="doc_header"><code>compute_Natural_cities</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>compute_Natural_cities</code>(**`points2_clusters`**, **\*\*`kwargs`**)

Compute Natural cities clustering

:param np.array points2_clusters: a (N,2) numpy array containing the obsevations

:returns: list with numpy arrays for all the clusters obtained

## Similarity form metric

The function use the *Jaccar* index to and the points in the clusterization to obtain a metric tha is use to compare the clusters on level basis.

In [37]:
#export
def SSM(list_poly_c_1,list_poly_c_2 ,**kwargs):
    """
    The function calculates the Similarity Form Measurment (SMF)
    between two clusterizations 
    
    :param: list of nodes with points and polygons 
    
    :param: list of nodes with points and polygons 
    
    :param bool verbose: To print to debugg
    
    :returns double: The similarity mesuarment
    """
    verbose= kwargs.get('verbose', False)
    ##### Get intersection 
    list_de=[]
    for i in list_poly_c_1:
        list_de.append([ i.polygon_cluster.intersection(  j.polygon_cluster ) for  j in list_poly_c_2])
    
    list_de_bool = []
    for i in list_de:
        list_de_bool.append([not j.is_empty for j in i])
    
    list_de_index = []
    #print(list_de_bool)
    for i in list_de_bool:
        if any(i):
            list_de_index.append(i.index(True))
        else:
            list_de_index.append(None)
    jacc_sim_po = []
    for num, node in enumerate(list_poly_c_1):
        ### ver eque pasa cuando se tienen 2 
        if list_de_index[num] is not None:
            node_get = list_poly_c_2[list_de_index[num]]
            poli_int = list_de[num][list_de_index[num]]
            
            ####Puntos en la interseccion
            #print(node)
            points_all = node.get_point_decendent()
            res_bool =[ poli_int.contains(p) for p in points_all] ### Como no necesito los puntos basta con esto
            card = sum(res_bool)
            ###Obtenemos jaccard 
            sim_jacc = (poli_int.area)/(node.polygon_cluster.area + node_get.polygon_cluster.area - poli_int.area)
            if verbose:
                print("jaccard: " ,sim_jacc)
                print("cardinal: " ,card )
            jacc_sim_po.append(sim_jacc* card)#####Cuando hay interseccion 
        else:
            jacc_sim_po.append(0) #### Cuando no hay
    
    arr_bool = np.array(list_de_bool)

    Q_not= []
    for col in range(arr_bool.shape[1]):
        cols_sel= arr_bool[:,col].any()
        if cols_sel ==False:
            Q_not.append(col)
    #print(Q_not)
    len_Q_not=[]
    if Q_not:
         len_Q_not =[len(list_poly_c_2[i].get_point_decendent())  for i in Q_not] 

    P_sum = sum([len(node.get_point_decendent())  for node in list_poly_c_1])
    deno =P_sum + sum(len_Q_not)
    return sum(jacc_sim_po)/deno

## Clustering an hieralchical structure

In [38]:
#export
def get_tree_from_clustering(cluster_tree_clusters):
     """ Returns the tree from the iterative clustering, the cluster_tree_cluster
     
     :param cluster_tree_clusters is a list of list with a dictionary that 
            should contain the point of the next level clusters, the name of the parent
            cluster of such clusters (name of the current node), and the point that 
            are consider noise. 
    
     :return A list of list that contains the nodes for each level of the tree. 
     """ 
     ##### La estructura de arbol 
     all_level_clusters =[]
     previus_level=[]
     list_len = len(cluster_tree_clusters)-1
     for level_num_clus, level_cluster in enumerate(cluster_tree_clusters):
          level_nodes=[]
          for cluster_te in level_cluster:
               node_l = NodeCluster(name= cluster_te['parent'])
               to_concat= [point_arr for point_arr in cluster_te['points']]
               if len(to_concat)>0: 
                    points_poly = np.concatenate(to_concat)
               else:
                    points_poly = np.array([], dtype= np.float64).reshape(0,2)
               
               points_poly = np.concatenate(
                                   (points_poly,cluster_te['noise_points']),
                                   axis=0
                              )
               
               node_l.polygon_cluster =  get_alpha_shape(points_poly)
               ##### Es necesario que si es el último nivel todos los puntoas sean
               ## considerados como ruido pues aunque se haya hecho la clusterizacion
               #  ya no se bajo al siguiente nivel
               ## 
               if level_num_clus==list_len:
                    node_l.point_cluster_noise = shapely.geometry.MultiPoint(points_poly) 
               else:
                    node_l.point_cluster_noise = shapely.geometry.MultiPoint(cluster_te['noise_points']) 
               pos=node_l.name.rfind('_L')
               if node_l.name[:pos]=='':
                    node_l.parent = None
               else:
                    lis_pa=[item  for item in previus_level if item.name == node_l.name[:pos]]
                    node_l.parent= lis_pa[0]
               level_nodes.append(node_l)
          all_level_clusters.append(level_nodes)
          previus_level = level_nodes
     return  all_level_clusters

In [39]:
#hide
#export

def generate_tree_clusterize_form(**kwargs ):
    """
    Generates all the experiment all the experiment creates the data and clusterize using the algorithm available
    
    :param levels_tree: Levels for the tree
    
    :param int per_cluster: Points per clusters
    
    :param levels_cluster:  Levels to clusterize
    
    :param bool verbose:  To print some outputs 
    
    :returns: a dictionary with all the  data frames and a dictionary 
    with the similarity measurment created
     """
    
    levels_tree= kwargs.get('tree_level', 4)
    per_cluster = kwargs.get('num_per_cluster', 200)
    levels_cluster = kwargs.get('levels_cluster', 4)
    verbose = kwargs.get('verbose', False)
    
    if verbose:
        print('generating tree')
    
    
    random.seed(int(time.time()))
    random_seed = random.randint(0,1500)
    print('Random to use: ',random_seed )
    print('With',levels_tree , ' levels' )
    tree_original= TreeClusters(levels_tree, random_seed= random_seed)
    tree_original.populate_tree(number_per_cluster=per_cluster, avoid_intersec= True)
    tree_original_points= tree_original.get_points_tree()
    X_2=np.array([[p.x,p.y] for p in tree_original_points])
    dic_points_ori={'points':[X_2], 'parent':''}
    if verbose:
        print('tree with: ', X_2.shape )
    
    while X_2.shape[0] < 2000:
        
        print('tree with too few elements to clusterize creating new tree')
        random.seed(int(time.time()))
        random_seed = random.randint(0,1500)
        print('Random to use: ',random_seed )
        tree_original= TreeClusters(levels_tree, random_seed= random_seed)
        tree_original.populate_tree(number_per_cluster=per_cluster, avoid_intersec= True)
        tree_original_points= tree_original.get_points_tree()
        X_2=np.array([[p.x,p.y] for p in tree_original_points])
        dic_points_ori={'points':[X_2], 'parent':''}
        
    
    
    if verbose:
        print('tree generated')
    
    
    if verbose:
        print('clusterize and creating the trees')
    
    tree_Natural_c = recursive_clustering_tree(dic_points_ori,
                                               levels_clustering = levels_cluster,
                                              algorithm = 'natural_cities'
                                              )
    tree_DBSCAN = recursive_clustering_tree(dic_points_ori,
                                               levels_clustering = levels_cluster,
                                              algorithm = 'dbscan')
    tree_HDBSCAN = recursive_clustering_tree(dic_points_ori,
                                               levels_clustering = levels_cluster,
                                              algorithm = 'hdbscan')
    tree_OPTICS= recursive_clustering_tree(dic_points_ori,
                                               levels_clustering = levels_cluster,
                                              algorithm = 'optics')
    tree_knee = recursive_clustering_tree(dic_points_ori,
                                               levels_clustering = levels_cluster,
                                              algorithm = 'adaptative_DBSCAN')
    if verbose:
        print('DONE clusterize and creating the trees')
    ######  get the points dataframe for each tree 
    data_fram_or = tree_original.get_dataframe_recursive_node_label(func_level_nodes = levels_from_strings)
    df_Natural = tree_Natural_c.get_dataframe_recursive_node_label()
    df_DBSCAN = tree_DBSCAN.get_dataframe_recursive_node_label()
    df_HDBSCAN = tree_HDBSCAN.get_dataframe_recursive_node_label()
    df_OPTICS = tree_OPTICS.get_dataframe_recursive_node_label()
    df_knee = tree_knee.get_dataframe_recursive_node_label()
    
    df_Natural.name='Natural_C'
    df_DBSCAN.name= 'DBSCAN'
    df_HDBSCAN.name= 'HDBSCAN'
    df_OPTICS.name= 'OPTICS'
    df_knee.name = 'knee'
    
    if verbose:
        print('Original size',data_fram_or.shape )
        print('Natural size',df_Natural.shape)
        print('DBSCAN size',df_DBSCAN.shape)
        print('HDBSCAN size',df_HDBSCAN.shape)
        print('OPTICS size',df_OPTICS.shape)
        print('adaptative_DBSCAN size',df_knee.shape)
    
    ######For each dataframe  
    if verbose:
        print('get dataframe Original')
    get_tag_level_df_labels(data_fram_or, levels_cluster)
    ###Natural Cities
    if verbose:
        print('get dataframe Natural cities')
        
    dic_final_levels_Natural_c = get_dics_labels(tree_original, tree_Natural_c, levels_cluster)
    dic_label_final_levels_Natural=[ {'level_ori':dic['level_ori'], 'dict':mod_cid_label(dic['dict']) } for dic in  dic_final_levels_Natural_c]
    get_tag_level_df_labels(df_Natural, levels_cluster)
    for dic in dic_label_final_levels_Natural[1:]: ## En el nivel 0 no tiene sentido
        tag_ori = dic['level_ori']
        dic_lev = dic['dict']
        retag_originals(data_fram_or,
                        df_Natural,
                        tag_ori,
                        tag_ori,#### Como se hizo con la misma funcion tienen las misma etiqueta 
                        dic_lev)
    
    
    ### DBSCAN
    if verbose:
        print('get dataframe DBSCAN')
        
    dic_final_levels_DBSCAN = get_dics_labels(tree_original, tree_DBSCAN, levels_cluster)
    dic_label_final_levels_DBSCAN=[ {'level_ori':dic['level_ori'], 'dict':mod_cid_label(dic['dict']) } for dic in  dic_final_levels_DBSCAN]
    get_tag_level_df_labels(df_DBSCAN, levels_cluster)
    for dic in dic_label_final_levels_DBSCAN[1:]: ## En el nivel 0 no tiene sentido
        tag_ori = dic['level_ori']
        dic_lev = dic['dict']
        retag_originals(data_fram_or,
                        df_DBSCAN,
                        tag_ori,
                        tag_ori,#### Como se hizo con la misma funcion tienen las misma etiqueta 
                        dic_lev)
    
    
    ##HDBSCAN
    if verbose:
        print('get dataframe HDBSCAN')
    dic_final_levels_HDBSCAN = get_dics_labels(tree_original, tree_HDBSCAN, levels_cluster)
    dic_label_final_levels_HDBSCAN=[ {'level_ori':dic['level_ori'], 'dict':mod_cid_label(dic['dict']) } for dic in  dic_final_levels_HDBSCAN]
    get_tag_level_df_labels(df_HDBSCAN, levels_cluster)
    for dic in dic_label_final_levels_HDBSCAN[1:]: ## En el nivel 0 no tiene sentido
        tag_ori = dic['level_ori']
        dic_lev = dic['dict']
        retag_originals(data_fram_or,
                        df_HDBSCAN,
                        tag_ori,
                        tag_ori,#### Como se hizo con la misma funcion tienen las misma etiqueta 
                        dic_lev)
    #### OPTICS
    if verbose:
        print('get dataframe OPTICS')
        
    dic_final_levels_OPTICS = get_dics_labels(tree_original, tree_OPTICS, levels_cluster)
    dic_label_final_levels_OPTICS=[ {'level_ori':dic['level_ori'], 'dict':mod_cid_label(dic['dict']) } for dic in  dic_final_levels_OPTICS]
    get_tag_level_df_labels(df_OPTICS, levels_cluster)
    for dic in dic_label_final_levels_OPTICS[1:]: ## En el nivel 0 no tiene sentido
        tag_ori = dic['level_ori']
        dic_lev = dic['dict']
        retag_originals(data_fram_or,
                        df_OPTICS,
                        tag_ori,
                        tag_ori,#### Como se hizo con la misma funcion tienen las misma etiqueta 
                        dic_lev)
    ##### Knee
    if verbose:
        print('get adaptative DBSCAN')
        
    dic_final_levels_knee = get_dics_labels(tree_original, tree_knee, levels_cluster)
    dic_label_final_levels_knee=[ {'level_ori':dic['level_ori'], 'dict':mod_cid_label(dic['dict']) } for dic in  dic_final_levels_knee]
    get_tag_level_df_labels(df_knee, levels_cluster)
    for dic in dic_label_final_levels_knee[1:]: ## En el nivel 0 no tiene sentido
        tag_ori = dic['level_ori']
        dic_lev = dic['dict']
        retag_originals(data_fram_or,
                        df_knee,
                        tag_ori,
                        tag_ori,#### Como se hizo con la misma funcion tienen las misma etiqueta 
                        dic_lev)
    ##############################################Get only signal  noise 
    data_fram_or_sig_noise = tree_original.get_tag_noise_signal_tree()
    df_Natural_sig_noise = tree_Natural_c.get_tag_noise_signal_tree()
    df_DBSCAN_sig_noise = tree_DBSCAN.get_tag_noise_signal_tree()
    df_HDBSCAN_sig_noise = tree_HDBSCAN.get_tag_noise_signal_tree()
    df_OPTICS_sig_noise = tree_OPTICS.get_tag_noise_signal_tree()
    df_knee_sig_noise = tree_knee.get_tag_noise_signal_tree()
    
    df_Natural_sig_noise.name='I_Natural_C'
    df_DBSCAN_sig_noise.name= 'I_DBSCAN'
    df_HDBSCAN_sig_noise.name= 'I_HDBSCAN'
    df_OPTICS_sig_noise.name= 'I_OPTICS'
    df_knee_sig_noise.name = 'I_Ada_DBSCAN'
    
    
    
    
    
    
    ######Evaluate form metric
    if verbose:
        print('Niveles: ' ,len(tree_original.levels_nodes))
        print('Nodos en el ultimo nivel: ' ,len(tree_original.levels_nodes[-1]))
        #print('tag_all : ' , data_fram_or['Final_tag'].unique())
    levels_r = range(0, levels_cluster)###
    resultado_form_metric = []
    for l in levels_r:
        
        d = { 'Level': l,
            'DBSCAN': SSM(tree_original.levels_nodes[l],
                                            tree_DBSCAN.levels_nodes[l]),
             'HDBSCAN': SSM(tree_original.levels_nodes[l],
                                             tree_HDBSCAN.levels_nodes[l]),
             'Natural': SSM(tree_original.levels_nodes[l],
                                             tree_Natural_c.levels_nodes[l]),
             'OPTICS': SSM(tree_original.levels_nodes[l],
                                            tree_OPTICS.levels_nodes[l]),
             'knee': SSM(tree_original.levels_nodes[l],
                                          tree_knee.levels_nodes[l])
             }
        resultado_form_metric.append(d)
    

    return {'Point_dataframes':{'original_retag':data_fram_or,
            'Natural_C':df_Natural ,
            'DBSCAN':df_DBSCAN,
            'HDBSCAN':df_HDBSCAN,
            'OPTICS':df_OPTICS,
            'knee':df_knee
           }, 'metric_form': resultado_form_metric,
            'Noise_signal':{
            'original_retag':data_fram_or_sig_noise,
            'Natural_C':df_Natural_sig_noise ,
            'DBSCAN':df_DBSCAN_sig_noise,
            'HDBSCAN':df_HDBSCAN_sig_noise,
            'OPTICS':df_OPTICS_sig_noise,
            'knee':df_knee_sig_noise
            }
           
           }


In [40]:
show_doc(generate_tree_clusterize_form)

<h4 id="generate_tree_clusterize_form" class="doc_header"><code>generate_tree_clusterize_form</code><a href="__main__.py#L4" class="source_link" style="float:right">[source]</a></h4>

> <code>generate_tree_clusterize_form</code>(**\*\*`kwargs`**)

Generates all the experiment all the experiment creates the data and clusterize using the algorithm available

:param levels_tree: Levels for the tree

:param int per_cluster: Points per clusters

:param levels_cluster:  Levels to clusterize

:param bool verbose:  To print some outputs 

:returns: a dictionary with all the  data frames and a dictionary 
with the similarity measurment created
 

In [41]:
show_doc(get_tree_from_clustering)

<h4 id="get_tree_from_clustering" class="doc_header"><code>get_tree_from_clustering</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>get_tree_from_clustering</code>(**`cluster_tree_clusters`**)

Returns the tree from the iterative clustering, the cluster_tree_cluster

:param cluster_tree_clusters is a list of list with a dictionary that 
       should contain the point of the next level clusters, the name of the parent
       cluster of such clusters (name of the current node), and the point that 
       are consider noise. 

:return A list of list that contains the nodes for each level of the tree. 

In [42]:
show_doc(SSM)

<h4 id="SSM" class="doc_header"><code>SSM</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>SSM</code>(**`list_poly_c_1`**, **`list_poly_c_2`**, **\*\*`kwargs`**)

The function calculates the Similarity Form Measurment (SMF)
between two clusterizations 

:param: list of nodes with points and polygons 

:param: list of nodes with points and polygons 

:param bool verbose: To print to debugg

:returns double: The similarity mesuarment

## Helper functions

These functions are use by the clustering algorithms and to create the structure, mosly by handly the names for each node 

In [43]:
show_doc(get_alpha_shape)

<h4 id="get_alpha_shape" class="doc_header"><code>get_alpha_shape</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>get_alpha_shape</code>(**`point_list`**)

Returns a polygon representing the hull of the points sample.

:param list point_list: list list of tuples with samples coordinates.

:returns shapely.Polygon: concave hull shapely polygon

In [44]:
show_doc(set_colinear)

<h4 id="set_colinear" class="doc_header"><code>set_colinear</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>set_colinear</code>(**`list_points`**)

Check if in the list of points any of triplet of points
is colinear
:param list list_points: List of shapely Points

:returns bool: True if all are not colinear 

In [45]:
show_doc(collinear)

<h4 id="collinear" class="doc_header"><code>collinear</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>collinear</code>(**`p1`**, **`p2`**, **`p3`**)

Check if the points are colinear 

:param shapely Point p1: point to chek if is colinear

:param shapely Point p2: point to chek if is colinear

:param shapely Point p3: point to chek if is colinear

:return bool: True if are colinear

In [46]:
show_doc(get_segments)

<h4 id="get_segments" class="doc_header"><code>get_segments</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>get_segments</code>(**`points`**)

Get the segments from a delaunay triangulation

:param points: Point to get Delaunay triangulation and exctract points 

:return edges: 

In [47]:
show_doc(get_polygons_buf)

<h4 id="get_polygons_buf" class="doc_header"><code>get_polygons_buf</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>get_polygons_buf</code>(**`lines`**)

Obtain the poligons from the lines

:param list lines: List of lines

:returns shapely polygon: the union of the union of 
edges (Polygon or multypolygon)

In [48]:
show_doc(labels_filtra)

<h4 id="labels_filtra" class="doc_header"><code>labels_filtra</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>labels_filtra</code>(**`point_points`**, **`multy_pol`**)

Labels the points in the multy_pol if no polygon contains 
a point is label as -1

:param shapely MultyPoint point_points: Points to check 

:param multy_pol

:returns np.array: Label array with -1 if is not contained 
in a polygon

In [ ]:
#hide
###Test
generate_tree_clusterize_form(tree_level= 2,per_cluster = 100, levels_cluster =  2,verbose = False)

Random to use:  332
With 2  levels
tree with too few elements to clusterize creating new tree
Random to use:  332


/Partition1/Cgeo/git/HierarchicalGeoClustering/HierarchicalGeoClustering/TreeClusters.py:96: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  return shapely.ops.cascaded_union(keep_triangles)
/Partition1/Cgeo/git/HierarchicalGeoClustering/HierarchicalGeoClustering/TreeClusters.py:377: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  points_cluster_o = [p for p in points_cluster if
/Partition1/Cgeo/git/HierarchicalGeoClustering/HierarchicalGeoClustering/TreeClusters.py:1086: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  all_points =all_points+[i for i in node.get_points(all_tag = iterative )]


tree with too few elements to clusterize creating new tree
Random to use:  1404
tree with too few elements to clusterize creating new tree
Random to use:  1404
tree with too few elements to clusterize creating new tree
Random to use:  1404
tree with too few elements to clusterize creating new tree
Random to use:  1404
tree with too few elements to clusterize creating new tree
Random to use:  1404
tree with too few elements to clusterize creating new tree
Random to use:  1404
tree with too few elements to clusterize creating new tree
Random to use:  1404
tree with too few elements to clusterize creating new tree
Random to use:  1404
tree with too few elements to clusterize creating new tree
Random to use:  1404
tree with too few elements to clusterize creating new tree
Random to use:  1404
tree with too few elements to clusterize creating new tree
Random to use:  1404
tree with too few elements to clusterize creating new tree
Random to use:  1404
tree with too few elements to clusterize